In [1]:
#pip install ./funda-scraper ipykernel weaviate-client

In [4]:
#docker compose up -d

In [ ]:
COLLECTION_DEF_FILE = 'weaviate/application/fundalytics/collection_def.json'

In [ ]:
import weaviate
import json

client = weaviate.connect_to_local()

try:

    current_collections = client.collections.list_all()

    with open(COLLECTION_DEF_FILE) as f:
        collection_def = json.load(f)

    if collection_def['class'] not in current_collections.keys():
        client.collections.create_from_dict(config=collection_def)

finally:
    client.close()